In [ ]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from konlpy.tag import Okt
okt = Okt()

## 전처리

In [ ]:
def textPreprocessing(text):
    text = okt.nouns(text)
    text = [w for w in text if not w in stop_words]
    text = " ".join(text)
    return text

In [ ]:
df = pd.read_json("kumohQnA.json")

stop_words = "좀 가 을 를 이 께서 에서 에 에게 께 한테 더러 에서 에게서 한테서 에 에서 로 으로 으로서 의 과 와 하고 보다 처럼 만큼 아 야 이여 여 만 도 은 는 조차 마저 까지 부터 이나 나 이나마 나마 이라도 라도 이야 야 이라야 라야 대해 하나 거나 위해 안녕하세요 건가 는걸 일이 가지 무엇 이면 안녕하십니까 안해 위해 던데 내야 등등 수가 또한 거나 이건 그거 거기 그것 려고 어케 안나 대로 여러분 치면 처럼 그게 어찌 저런 된거 푸나 끼리 고해 다가 로만 일도 이란 따라서 이기 이나 님들 여야 번은 이구 라서 정이 번만 런가 하니 워낙 아마 안이 부터"

In [ ]:
df['cleaned'] = df['question'].apply(textPreprocessing)

print('전체 문서의 수 :',len(df))
df['cleaned'].replace('', np.nan, inplace=True)
df = df[df['cleaned'].notna()]
print('전체 문서의 수 :',len(df))

corpus = []
for words in df['cleaned']:
    corpus.append(words.split())
print(corpus)

## Fine Tuning

In [ ]:
model = gensim.models.Word2Vec.load('C:\ko\ko.bin')
model.wv.save_word2vec_format("ko.bin.gz", binary = False)

In [ ]:
word2vec_model = Word2Vec(size = 200, window=5, min_count = 3, workers = 4, sg=1)
word2vec_model.build_vocab(corpus)
word2vec_model.intersect_word2vec_format("ko.bin.gz", lockf=1.0, binary=False)
word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs = 30)

In [ ]:
word2vec_model.wv.most_similar('동아리')

## 문장 벡터

In [ ]:
def vectors(document_list) :
    document_embedding_list = []
    
    for text in document_list:
        count = 0
        doc2vec = None
        for word in text.split():
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해단 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None : 
                    doc2vec = word2vec_model.wv[word]
                else : 
                    doc2vec = doc2vec + word2vec_model.wv[word]
                
        if doc2vec is not None :
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec/count
            document_embedding_list.append(doc2vec)
            
    return document_embedding_list

In [ ]:
document_embedding_list = vectors(df['cleaned'])
print('문서 벡터의 수 :', len(document_embedding_list))

## 사용자 입력 전처리 및 임베딩

In [ ]:
def text_vectors(text) :
    text_embedding_list = []
    
    count = 0
    doc2vec = None
    for word in text.split():
        if word in word2vec_model.wv.vocab:
            count += 1
            # 해단 문서에 있는 모든 단어들의 벡터값을 더한다.
            if doc2vec is None : 
                doc2vec = word2vec_model.wv[word]
            else : 
                doc2vec = doc2vec + word2vec_model.wv[word]

    if doc2vec is not None :
        # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
        doc2vec = doc2vec/count
        text_embedding_list.append(doc2vec)
            
    return text_embedding_list

In [ ]:
user_input = input()

In [ ]:
text = textPreprocessing(user_input)
print(text)
text_embedding = text_vectors(text)

## 코사인 유사도 매트릭스

In [ ]:
cosine_similarities = cosine_similarity(document_embedding_list, text_embedding)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

sim_scores = list(enumerate(cosine_similarities))
sim_scores = sorted(sim_scores, key=lambda x : x[1], reverse=True)
sim_scores = sim_scores[:5]

indices= [i[0] for i in sim_scores] # 가장 유사한 질문 5개의 인덱스

## 결과

In [ ]:
QnA = df[['question', 'answer']]
result = QnA.iloc[indices]
result